<a href="https://colab.research.google.com/github/Paulo-vitorCS/DeepLearning/blob/main/Projeto_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projeto 03: Classificação binária breast cancer com tuning dos parâmetros

###Etapa 01: Importação das bibliotecas

In [ ]:
!pip install skorch==0.8.0

     |████████████████████████████████| 113 kB 5.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn 
from skorch import NeuralNetBinaryClassifier  # Classificador para problemas binários
import torch
import torch.nn.functional as F
import seaborn as sns
from sklearn.model_selection import GridSearchCV  # Bib para Validação cruzada 

###Etapa 02: Base de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('/content/entradas_breast.csv')
classe = pd.read_csv('/content/saidas_breast.csv')

In [ ]:
# sklearn exige o formato numpy array, não tensores
previsores = np.array(previsores, dtype='float32')
classe = np.array(classe, dtype='float32').squeeze(1)  # Retira o 1 de (569, 1)

In [ ]:
previsores.shape

(569, 30)

In [ ]:
classe.shape

(569,)

###Etapa 03: Classe para a estrutura da rede neural

In [ ]:
class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer):
        super().__init__()
        # 30 -> 16 -> 16 -> 1
        self.dense0 = nn.Linear(30, neurons)  # Primeira camada densa
        initializer(self.dense0.weight)
        self.activation0 = activation 
        self.dense1 = nn.Linear(neurons, neurons)  # Segunda camada densa
        initializer(self.dense1.weight)
        self.activation1 = activation
        self.dense2 = nn.Linear(neurons, 1)  # Terceira camada densa
        initializer(self.dense2.weight)
        self.output = nn.Sigmoid()

    def forward(self,X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dense2(X)
        X = self.output(X)
        return X

###Etapa 04: Skorch

In [ ]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr=0.001,
                                                  optimizer__weight_decay=0.0001,
                                                  train_split=False)

###Etapa 05: Tuning dos parâmetros

In [ ]:
params = {'batch_size': [10, 30],  # É possível passar os valores ou funções as quais quero para o teste dentro das colchetes  
          'max_epochs': [50, 100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCELoss, torch.nn.HingeEmbeddingLoss],
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [16],
          'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_]}

In [ ]:
params

{'batch_size': [10, 30],
 'criterion': [torch.nn.modules.loss.BCELoss,
  torch.nn.modules.loss.HingeEmbeddingLoss],
 'max_epochs': [50, 100],
 'module__activation': [<function torch.nn.functional.relu>,
  <function torch.nn.functional.tanh>],
 'module__initializer': [<function torch.nn.init.uniform_>,
  <function torch.nn.init.normal_>],
 'module__neurons': [8, 16],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD]}

In [ ]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring='accuracy', cv=2)   
grid_search = grid_search.fit(previsores, classe)  # Cria as estruturas de redes neurais pra conseguir fazer os testes com os parametros

  epoch    train_loss     dur
-------  ------------  ------
      1       37.3239  0.2294
      2       37.3239  0.0569
      3       37.3239  0.0410
      4       37.3239  0.0437
      5       37.3239  0.0417
      6       37.3239  0.0410
      7       37.3239  0.0492
      8       37.3239  0.0421
      9       37.3239  0.0505
     10       37.3239  0.0435
     11       37.3239  0.0412
     12       37.3239  0.0441
     13       37.3239  0.0433
     14       37.3239  0.0436
     15       37.3239  0.0489
     16       37.3239  0.0424
     17       37.3239  0.0425
     18       37.3239  0.0459
     19       37.3239  0.0439
     20       37.3239  0.0494
     21       37.3239  0.0431
     22       37.3239  0.0424
     23       37.3239  0.0486
     24       37.3239  0.0422
     25       37.3239  0.0627
     26       37.3239  0.0417
     27       37.3239  0.0454
     28       37.3239  0.0484
     29       37.3239  0.0416
     30       37.3239  0.0588
     31       37.3239  0.0404
     32   

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      5        1.5945  0.0527
      6        1.4982  0.0417
      7        1.3688  0.0416
      8        1.1830  0.0406
      9        0.9562  0.0428
     10        0.7782  0.0497
     11        0.7015  0.0514
     12        0.6792  0.0477
     13        0.6731  0.0430
     14        0.6712  0.0477
     15        0.6704  0.0435
     16        0.6699  0.0461
     17        0.6696  0.0434
     18        0.6694  0.0417
     19        0.6692  0.0415
     20        0.6690  0.0423
     21        0.6689  0.0417
     22        0.6688  0.0429
     23        0.6687  0.0480
     24        0.6686  0.0434
     25        0.6685  0.0425
     26        0.6685  0.0412
     27        0.6684  0.0402
     28        0.6684  0.0474
     29        0.6683  0.0488
     30        0.6683  0.0421
     31        0.6682  0.0422
     32        0.6682  0.0600
     33        0.6681  0.0437
     34        0.6681  0.0429
     35        0.6681  0.0435
     36        0.6681  0.0416
     37        0.6680  0.0410
     38   

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      5        1.4863  0.0542
      6        1.4141  0.0470
      7        1.3348  0.0431
      8        1.2392  0.0439
      9        1.1165  0.0437
     10        0.9710  0.0490
     11        0.8322  0.0439
     12        0.7363  0.0429
     13        0.6911  0.0442
     14        0.6751  0.0500
     15        0.6698  0.0450
     16        0.6679  0.0440
     17        0.6670  0.0434
     18        0.6666  0.0435
     19        0.6663  0.0573
     20        0.6660  0.0519
     21        0.6659  0.0450
     22        0.6657  0.0464
     23        0.6656  0.0425
     24        0.6655  0.0426
     25        0.6655  0.0505
     26        0.6654  0.0465
     27        0.6653  0.0442
     28        0.6653  0.0446
     29        0.6652  0.0448
     30        0.6652  0.0524
     31        0.6652  0.0439
     32        0.6651  0.0444
     33        0.6651  0.0452
     34        0.6651  0.0472
     35        0.6650  0.0453
     36        0.6650  0.0533
     37        0.6650  0.0512
     38   

...

      7        0.6178  0.0306
      8        0.5897  0.0243
      9        0.5656  0.0262
     10        0.5438  0.0243
     11        0.5251  0.0249
     12        0.5081  0.0276
     13        0.4957  0.0246
     14        0.4828  0.0280
     15        0.4718  0.0246
     16        0.4631  0.0300
     17        0.4556  0.0235
     18        0.4498  0.0285
     19        0.4452  0.0255
     20        0.4413  0.0237
     21        0.4374  0.0237
     22        0.4342  0.0295
     23        0.4318  0.0308
     24        0.4298  0.0244
     25        0.4279  0.0249
     26        0.4261  0.0264
     27        0.4245  0.0252
     28        0.4230  0.0287
     29        0.4215  0.0260
     30        0.4199  0.0262
     31        0.4186  0.0240
     32        0.4174  0.0268
     33        0.4162  0.0231
     34        0.4147  0.0264
     35        0.4151  0.0279
     36        0.4141  0.0237
     37        0.4117  0.0238
     38        0.4108  0.0266
     39        0.4098  0.0238
     40   

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      8        0.4869  0.0208
      9        0.4867  0.0258
     10        0.4865  0.0245
     11        0.4864  0.0206
     12        0.4862  0.0187
     13        0.4861  0.0202
     14        0.4859  0.0235
     15        0.4858  0.0252
     16        0.4856  0.0232
     17        0.4855  0.0195
     18        0.4853  0.0200
     19        0.4851  0.0198
     20        0.4850  0.0205
     21        0.4848  0.0198
     22        0.4847  0.0194
     23        0.4845  0.0224
     24        0.4844  0.0196
     25        0.4842  0.0199
     26        0.4841  0.0203
     27        0.4839  0.0198
     28        0.4837  0.0198
     29        0.4836  0.0202
     30        0.4834  0.0198
     31        0.4833  0.0194
     32        0.4831  0.0236
     33        0.4830  0.0199
     34        0.4828  0.0229
     35        0.4827  0.0236
     36        0.4825  0.0194
     37        0.4823  0.0256
     38        0.4822  0.0197
     39        0.4820  0.0233
     40        0.4819  0.0190
     41   

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      8        0.5533  0.0243
      9        0.5503  0.0210
     10        0.5473  0.0205
     11        0.5445  0.0207
     12        0.5419  0.0217
     13        0.5393  0.0153
     14        0.5368  0.0296
     15        0.5344  0.0257
     16        0.5321  0.0216
     17        0.5299  0.0194
     18        0.5278  0.0195
     19        0.5257  0.0196
     20        0.5237  0.0198
     21        0.5218  0.0195
     22        0.5199  0.0202
     23        0.5181  0.0225
     24        0.5164  0.0202
     25        0.5147  0.0206
     26        0.5130  0.0191
     27        0.5114  0.0211
     28        0.5098  0.0212
     29        0.5083  0.0219
     30        0.5068  0.0228
     31        0.5054  0.0250
     32        0.5040  0.0228
     33        0.5026  0.0226
     34        0.5013  0.0192
     35        0.5000  0.0232
     36        0.4987  0.0195
     37        0.4974  0.0164
     38        0.4962  0.0230
     39        0.4950  0.0212
     40        0.4911  0.0208
     41   

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [ ]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init.uniform_>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
melhor_precisao

0.747133679268594